In [1]:
import pandas as pd
from scripts.report.ecf.ecf import ECFReport

In [2]:
SORT_KEY = ['datetime','programme','donor']

In [3]:
def clean_source(df,m,y):
    period = "{}/{}".format(m,y)
    df.datetime = pd.to_datetime(df.datetime)
    df.sort_values(by=['datetime','programme','donor'], inplace=True)
    df.set_index('datetime').truncate(before='04/2016', after='04/2016').reset_index()
    df.fillna(0,inplace=True)
    return df

In [4]:
opts = {
    'ONLY_NGO' : ['FoodLink'],
    'ONLY_STAGES' : ['collection']
}
report = ECFReport(**opts)

In [5]:
programmes = [u'ECF Van 01', u'ECF Van 03', u'ECF Van 02']
report.generate_single_report('FoodLink', 'collection', programmes);

Report generating to: data/Report/FoodLink.2016.4.collection.ecf.report.xlsx
Done!


In [6]:
fns = report.available_csvs()

In [7]:
df = report.map_source_to_dataframe(fns)

In [8]:
# TARGET
df.ix[:,:'datetime'].sum().sum()

7727.4999999999982

In [9]:
# Total Rows - Wide Format
df.datetime.count()

579

In [10]:
# Rows per Day - Wide Format
df.datetime.value_counts()

2016-04-26    33
2016-04-15    33
2016-04-13    33
2016-04-12    33
2016-04-27    33
2016-04-22    32
2016-04-08    32
2016-04-25    32
2016-04-29    31
2016-04-18    31
2016-04-07    31
2016-04-20    31
2016-04-19    30
2016-04-14    30
2016-04-11    29
2016-04-06    28
2016-04-21    27
2016-04-05    25
2016-04-28    25
Name: datetime, dtype: int64

In [11]:
dfs.head(5)

,datetime,donor,programme,variable,value
450,2016-04-25,Disneyland,ECF Van 03,Candies,5.0
579,2016-04-05,CityGarden,ECF Van 01,Cooked,7.0
580,2016-04-05,FourSeasons,ECF Van 01,Cooked,91.0
581,2016-04-05,GrandHyatt,ECF Van 01,Cooked,6.0
582,2016-04-05,HKClub,ECF Van 01,Cooked,16.0


In [ ]:
# Total Rows - Long Format
dfs.datetime.count()

In [12]:
# Rows per Day - Long Format
dfs.datetime.value_counts()

2016-04-26    38
2016-04-15    38
2016-04-25    38
2016-04-12    38
2016-04-22    37
2016-04-18    37
2016-04-27    36
2016-04-13    36
2016-04-20    36
2016-04-29    35
2016-04-07    35
2016-04-11    34
2016-04-19    34
2016-04-08    34
2016-04-14    33
2016-04-21    32
2016-04-06    30
2016-04-05    29
2016-04-28    27
Name: datetime, dtype: int64

In [13]:
dfs = dfs.sort_values(by=SORT_KEY)

In [14]:
dfs.datetime = dfs.datetime.dt.strftime('%d-%b-%y')

In [15]:
dfs = dfs.set_index('datetime')

In [16]:
dfs.columns = ['donor','programme','category','kg']

In [17]:
# dfs.set_index('total', append=True, inplace=True)

### Merge in Donors

In [18]:
df_donor = report.map_donors_to_dataframe(fns)

In [19]:
df_donor.head()

,id,name_en,location,programme
0,Parkview,Hong Kong Parkview,"Tai Tam Reservoir Road, Hong Kong",ECF Van 01
1,AMC,Aberdeen Marina Club,"8 Shum Wan Road, Sham Wan, Hong Kong",ECF Van 01
2,IslandSouth,L’hotel Island South Hong Kong,"55 Wong Chuk Hang Road, Aberdeen, Hong Kong",ECF Van 01
3,GrandHyatt,Grand Hyatt Hong Kong,"1 Harbour Road, Hong Kong",ECF Van 01
4,Novotel,Novotel Century Hong Kong,"238 Jaffe Road, Wanchai, Hong Kong",ECF Van 01


In [20]:
dfs.groupby(level='datetime').sum().head(5)

,kg
datetime,
05-Apr-16,496.5
06-Apr-16,297.8
07-Apr-16,383.0
08-Apr-16,558.8
11-Apr-16,393.7


In [21]:
df_merge = dfs.reset_index().merge(df_donor,how='left',left_on=['donor','programme'],right_on=['id','programme'])
cols = ['datetime','programme','name_en','location','category','kg']
dfx = df_merge[cols]
dfx.columns = ['datetime','programme','donor','address','category','kg']
dfx = dfx.sort_values(by=SORT_KEY)
dfx.set_index(SORT_KEY, inplace=True)
dfx.head(5)

address  \
datetime  programme  donor                                                                             
05-Apr-16 ECF Van 01 City Garden Hotel                    9 City Garden Road, North Point, Hong Kong   
                     Four Seasons Hotel Hong Kong               8 Finance Street, Central, Hong Kong   
                     Four Seasons Hotel Hong Kong               8 Finance Street, Central, Hong Kong   
                     Grand Hyatt Hong Kong                                 1 Harbour Road, Hong Kong   
                     Hotel LKF by Rhombus          33 Wyndham Street, Lan Kwai Fong, Central, Hon...   

                                                    category    kg  
datetime  programme  donor                                          
05-Apr-16 ECF Van 01 City Garden Hotel                Cooked   7.0  
                     Four Seasons Hotel Hong Kong     Cooked  91.0  
                     Four Seasons Hotel Hong Kong  Trimmings  24.0  
                     Grand Hyatt Hong Kong            Cooked   6.0  
                     Hotel LKF by Rhombus             Cooked   4.0

### Missing Values

In [22]:
# Everything still there? Original
dfs['kg'].sum()

7727.5

In [23]:
# Everything still there? New
dfx['kg'].sum()

7727.5

In [24]:
# Total Rows - Long Format - Original
dfs.reset_index().datetime.count()

657

In [25]:
# Total Rows - Long Format - New
dfx.reset_index().datetime.count()

657

In [26]:
dfx[dfx.isnull().any(axis=1)]

,,,address,category,kg
datetime,programme,donor,,,


In [27]:
# df_donor[df_donor.id == 'Regalairport']

In [28]:
# dfs[dfs.donor == 'Regalairport']

In [29]:
# dfs.donor.value_counts()

In [30]:
# dfs[(dfs.programme == 'ECF Van 03') & (dfs.donor == 'Regalairport')]

In [31]:
# df[df.donor == 'Regalairport']

### Total

In [32]:
dfx.head(5)

address  \
datetime  programme  donor                                                                             
05-Apr-16 ECF Van 01 City Garden Hotel                    9 City Garden Road, North Point, Hong Kong   
                     Four Seasons Hotel Hong Kong               8 Finance Street, Central, Hong Kong   
                     Four Seasons Hotel Hong Kong               8 Finance Street, Central, Hong Kong   
                     Grand Hyatt Hong Kong                                 1 Harbour Road, Hong Kong   
                     Hotel LKF by Rhombus          33 Wyndham Street, Lan Kwai Fong, Central, Hon...   

                                                    category    kg  
datetime  programme  donor                                          
05-Apr-16 ECF Van 01 City Garden Hotel                Cooked   7.0  
                     Four Seasons Hotel Hong Kong     Cooked  91.0  
                     Four Seasons Hotel Hong Kong  Trimmings  24.0  
                     Grand Hyatt Hong Kong            Cooked   6.0  
                     Hotel LKF by Rhombus             Cooked   4.0

In [33]:
import xlsxwriter

In [34]:
writer = pd.ExcelWriter('food.xlsx', engine='xlsxwriter')
pd.DataFrame(['placeholder']).to_excel(writer, 'food', startrow=0, index=False, header=False, )
dfx.to_excel(writer, 'food', startrow=1)
workbook = writer.book
worksheet = writer.sheets['food']
worksheet.merge_range('A1:F1', '2016 ties (Cookeddwqdqdwdwq  Food) - January')

In [35]:
offset = 2
totals = dfs.groupby(level='datetime').sum()
rowspan = dfs.ix[:,-1:].groupby(level='datetime').count()
totals['span'] = rowspan
totals['row_end'] = (totals['span'].cumsum() + offset).astype('int')
# totals['row_end'] = totals['row_end'] - range(1,len(totals)+1)
totals['row_start'] = (totals['row_end'].shift().fillna(offset).astype('int') + 1).astype('int')

In [36]:
totals['row_end'].head(5)

datetime
05-Apr-16     31
06-Apr-16     61
07-Apr-16     96
08-Apr-16    130
11-Apr-16    164
Name: row_end, dtype: int64

In [37]:
totals['row_start'].head(5)

datetime
05-Apr-16      3
06-Apr-16     32
07-Apr-16     62
08-Apr-16     97
11-Apr-16    131
Name: row_start, dtype: int64

In [38]:
format = workbook.add_format()
format.set_align('center')
format.set_align('bottom')

In [39]:
for i, r in totals.iterrows():
    print i,':', int(r.row_start), int(r.row_end), r.kg
    try:
        worksheet.merge_range("G{row_start}:G{row_end}".format(**r), r.kg, format)
    except UserWarning as e:
        print e
        worksheet.write_number("G{row_start}".format(**r), r.kg, format)
        

05-Apr-16 : 3 31 496.5
06-Apr-16 : 32 61 297.8
07-Apr-16 : 62 96 383.0
08-Apr-16 : 97 130 558.8
11-Apr-16 : 131 164 393.7
12-Apr-16 : 165 202 424.9
13-Apr-16 : 203 238 461.0
14-Apr-16 : 239 271 318.9
15-Apr-16 : 272 309 393.0
18-Apr-16 : 310 346 432.8
19-Apr-16 : 347 380 389.5
20-Apr-16 : 381 416 383.8
21-Apr-16 : 417 448 343.7
22-Apr-16 : 449 485 441.7
25-Apr-16 : 486 523 474.2
26-Apr-16 : 524 561 432.2
27-Apr-16 : 562 597 400.9
28-Apr-16 : 598 624 319.4
29-Apr-16 : 625 659 381.7


In [40]:
col_widths = [15,15,45,60,10,5,10]

format = workbook.add_format()
format.set_align('center')
format.set_align('vcenter')
format.set_font('Courier New')

for col, width in enumerate(col_widths):
    worksheet.set_column(col, col, width, cell_format=format) 

In [41]:
writer.save()